# Override Driving

Für Pre- oder Postprocessing (also alles was vor oder nach dem Ausführen des Modelles kommt) kann die Datei `override_driving.py` angepasst werden und zusammen mit dem Modell im Auto hochgeladen werden

## Anwendungsbeispiele

Als Idee

- Eine Farbfilter einbauen (z.B. nur blauer Farbkanal geht ins Model)
    - Natürlich muss dieser Filter auch beim Training eingebaut werden
- Steering oder Throttle nachbearbeiten (ungültige oder extreme Werte abfangen)
  - Z.B. Throttle muss immer >0.0 sein
- Throttle auf einen fixen Wert setzten und nur Steering trainieren*
- Eine Serie von Bilder ins Model geben anstatt nur ein Bild (anspruchsvoll)
- ...

*bei der Challenge wird getestet, wie gut das Modell die Bahn abfährt und wie schnell es dabei ist.

## Inhalt und Funktionsweise von `override_driving.py`

Auf dem Auto wird für jedes Kamerabild die Methode `predict_throttle_speed(image)` aufgerufen, welche als Rückgabewert `return throttle, steering` liefern muss. Alles andere kann grundsätzlich angepasst werden.

Inhalt vom original `override_driving.py`:

In [ ]:
import os
from pathlib import Path
import onnxruntime as ort
from torchvision import transforms

# HINT Diese Libraries sind auf dem Raspberry Pi installiert und dürfen daher verwendet werden
# import numpy as np
# import torch
# import cv2
# from PIL import Image
# Alle Python-Standardlibs können auch verwendet werden

convert_tensor = transforms.ToTensor()

path = Path(os.path.dirname(os.path.realpath(__file__)))

model = path / 'self_driving_model.onnx' # HINT Modelname so ändern wie dein Modell heisst
ORT_SESSION = ort.InferenceSession(str(model), providers=['CPUExecutionProvider'])
idx = 0

def predict_next_input(image):
    global ORT_SESSION
    tensor_img = convert_tensor(image).unsqueeze(0).numpy()
    result = ORT_SESSION.run(None, {'image': tensor_img})
    steering_data = float(result[0][0][1])
    throttle_data = float(result[0][0][0])
    return throttle_data, steering_data


def predict_throttle_speed(image): # HINT Methodenname&Parameter nicht verändern. Diese wird vom Raspberry Pi aufgerufen
    throttle, steering = predict_next_input(image)
    return throttle, steering # HINT Rückgabe muss genau so sein


### Override Driving Vorlage mit Blaufilter

zu verwenden mit `06d_Fahrmodell_Training_mit_Blaufilter.ipynb`. Denke daran, die gleichen Werte bei den Filter zu setzten wie beim Training!

In [ ]:
import os
from pathlib import Path
import onnxruntime as ort
from torchvision import transforms

import numpy as np
import cv2

convert_tensor = transforms.ToTensor()

path = Path(os.path.dirname(os.path.realpath(__file__)))

model = path / 'self_driving_model_blaufilter.onnx' # HINT Modelname so ändern wie dein Modell heisst
ORT_SESSION = ort.InferenceSession(str(model), providers=['CPUExecutionProvider'])
idx = 0


def predict_next_input(image):
    global ORT_SESSION
    image = filter_image(image)
    tensor_img = convert_tensor(image).unsqueeze(0).numpy()
    result = ORT_SESSION.run(None, {'image': tensor_img})
    steering_data = result[0][0][1]
    throttle_data = result[0][0][0]
    return throttle_data, steering_data


def predict_throttle_speed(image): # HINT Methodenname&Parameter nicht verändern. Diese wird vom Raspberry Pi aufgerufen
    throttle, steering = predict_next_input(image)
    return throttle, steering # HINT Rückgabe muss genau so sein



def filter_image(pil_image):

    # Bild als array laden und in korrektes Format bringen
    image_array = np.array(pil_image)
    image_array *= 255
    image_array = 255 - cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)

    # Color Filter
    lower1 = np.array([0, 75, 115]) # <- Werte können angepasst werden
    upper1 = np.array([115, 205, 255]) # <- Werte können angepasst werden
    color_filtered = cv2.inRange(image_array, lower1, upper1)

    # Kleinere Pixelbereiche entfernen
    kernel = np.ones((2, 2), np.uint8) # <- Werte können angepasst werden, z.B. (4,4)
    dilated_mask = cv2.erode(color_filtered, kernel, iterations=1)

    # Oberer Bildbereich abschneiden
    dilated_mask[:70, :] = 0.0 # <- Wert kann angepasst werden, z.B. [:50, :]

    # Debug-Ausgaben (auskommentieren, wenn ihr die Bilder im Model-Zip speichern wollt)
    # Die Bilder landen im .zip und ihr könnt im Webinterface einfach das zip heruterladen
    # bgr_image = cv2.cvtColor(dilated_mask, cv2.COLOR_GRAY2BGR)
    # cv2.imwrite(str(path / f"output_image_{idx}_maks.png"), bgr_image)
    # cv2.imwrite(str(path / f"output_image_{idx}_color.png"), image_array)
    # np.save(str(path / f"./image_array_{idx}.np"),image_array)

    return dilated_mask

## Eigenes `override_driving.py` auf Auto laden

Das Auto akzeptiert beim hochladen des Models entweder ein einfach `*.onnx` Model oder, wenn ihr `override_driving.py` benützten wollt, ein `*.zip`, welches das `override_driving.py` sowie ein Modell enthält.

**Beispiel:**
- `model_v0.zip` enthält:
  - `override_driving.py` (muss genau so heissen)
  - `self_driving_model.py` (muss so heissen, wie im `override_driving.py` angegeben)